In [21]:
!pip install lightfm

Processing ./.cache/pip/wheels/f0/cd/a5/b07914aa223c05ed61880d4c59f64a7febf117dbd2c2cbcf49/lightfm-1.15-cp37-cp37m-linux_x86_64.whl


In [2]:
import pandas as pd
import numpy as np

In [430]:
df_cliente_produto = pd.read_csv('Dataset-1-v2.csv')
df_page_views = pd.read_csv('Dataset-2.csv', )
df_produtos = pd.read_csv('Dataset-3.csv')

In [431]:
# Junta todas as respostas em uma só coluna

respostas_col = [col for col in df_cliente_produto if col.startswith('resposta')]
df_cliente_produto[respostas_col] = df_cliente_produto[respostas_col].astype(str)
df_cliente_produto['respostas'] = df_cliente_produto[respostas_col].apply(
    lambda x: '|'.join(x), axis=1)

In [432]:
# Selecionando apenas as colunas relativas ao perfil do cliente

df_clientes = df_cliente_produto[['userid', 'idade', 'genero', 'estadocivil', 'billingcity',
       'profissaopessoa', 'patrimonioautomoveis', 'patrimonioimoveis',
       'patrimonioinvestimentos', 'patrimoniooutros', 'produto',
       'valorrendimento', 'dataefetivacao', 'nivelconhecimentoatual',
       'scorenivelconhecimento', 'perfilinvestidor', 'rendamensal',
       'valorpatrimonio', 'respostas', 'scorerisco', 'scoreobjetivos',
       'scoresituacaofinanceira', 'createddate', 'dataclienteefetivado']]

In [433]:
df_clientes.shape

(52857, 24)

In [434]:
# Removendo os perfis duplicados

df_clientes = df_clientes[~df_clientes['userid'].duplicated()]

In [435]:
df_clientes.shape

(13315, 24)

In [436]:
df_produtos.shape

(8603, 86)

In [437]:
def generate_int_id(dataframe, id_col_name):
    """
    Generate unique integer id for clients and products

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    id_col_name : String 
        New integer id's column name.
        
    Returns
    -------
    Dataframe
        Updated dataframe containing new id column 
    """
    new_dataframe=dataframe.assign(
        int_id_col_name=np.arange(len(dataframe))
        ).reset_index(drop=True)
    return new_dataframe.rename(columns={'int_id_col_name': id_col_name})

In [438]:
df_clientes = generate_int_id(df_clientes, 'cliente_id_num')
df_produtos = generate_int_id(df_produtos, 'produto_id_num')

In [439]:
# atribuindo peso 1 para todos os relacionamentos existentes (cliente comprou o produto)

df_cliente_produto['weight'] = 1

In [441]:
print(df_cliente_produto.shape)
print(len(df_cliente_produto['userid'].unique()))
print(len(df_cliente_produto['produto'].unique()))

(52857, 62)
13315
1027


In [442]:
# Populando cliente_id_num no dataset com os relacionamentos

df_cliente_produto = pd.merge(
    df_cliente_produto, 
    df_clientes[['userid','cliente_id_num']],
    on='userid', 
    how='left'
)

In [443]:
print(df_cliente_produto.shape)
print(len(df_cliente_produto['userid'].unique()))
print(len(df_cliente_produto['produto'].unique()))

(52857, 63)
13315
1027


In [444]:
# Populando produto_id_num no dataset com os relacionamentos

df_cliente_produto = pd.merge(
    df_cliente_produto, 
    df_produtos[['ProdutoId','produto_id_num']],
    left_on='produto', 
    right_on='ProdutoId',
    how='outer'
)

In [445]:
print(df_cliente_produto.shape)
print(len(df_cliente_produto['userid'].unique()))
print(len(df_cliente_produto['produto'].unique()))

(60560, 65)
13316
1028


In [60]:
def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping 

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    features_name : List
        List of feature columns name avaiable in dataframe. 
        
    Returns
    -------
    List of all features for mapping 
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features

def create_features(dataframe, features_name, id_col_name):
    """
    Generate features that will be ready for feeding into lightfm

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe which contains features
    features_name : List
        List of feature columns name avaiable in dataframe
    id_col_name: String
        Column name which contains id of the question or
        answer that the features will map to.
        There are two possible values for this variable.
        1. questions_id_num
        2. professionals_id_num

    Returns
    -------
    Pandas Series
        A pandas series containing process features
        that are ready for feed into lightfm.
        The format of each value
        will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
        Ex. -> (1, ['military', 'army', '5'])
    """

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features

In [283]:
cliente_feature_list = generate_feature_list(df_clientes, ['perfilinvestidor'])

In [284]:
produto_feature_list = generate_feature_list(df_clientes, ['perfilinvestidor'])

In [87]:
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import auc_score

In [286]:
dataset = Dataset()
dataset.fit(
    set(df_clientes['cliente_id_num']), 
    set(df_produtos['produto_id_num']),
    item_features=cliente_feature_list, 
    user_features=produto_feature_list)

In [66]:
data = {'id_cliente':[1, 2, 3, 4] , 'nome':['Tom', 'Jack', 'nick', 'juli'], 'perfil':[1, 2, 3, 5]} 
cli = pd.DataFrame(data) 

In [67]:
data = {'id_produto':[1, 2, 3, 4] , 'nome':['CDB', 'LCI', 'LCA', 'FII'], 'risco':[1, 2, 3, 5]} 
prod = pd.DataFrame(data) 

In [68]:
cliente_feature_list = generate_feature_list(cli, ['perfil'])
produto_feature_list = generate_feature_list(prod, ['risco'])

In [69]:
cli['key'] = 0
prod['key'] = 0

df_merge = cli.merge(prod, on='key')

In [70]:
from random import randrange, uniform, randint

df_merge['weight'] = np.random.randint(0, 2, df_merge.shape[0])

In [71]:
df_merge

,id_cliente,nome_x,perfil,key,id_produto,nome_y,risco,weight
0,1,Tom,1,0,1,CDB,1,0
1,1,Tom,1,0,2,LCI,2,1
2,1,Tom,1,0,3,LCA,3,0
3,1,Tom,1,0,4,FII,5,0
4,2,Jack,2,0,1,CDB,1,0
5,2,Jack,2,0,2,LCI,2,1
6,2,Jack,2,0,3,LCA,3,0
7,2,Jack,2,0,4,FII,5,1
8,3,nick,3,0,1,CDB,1,1
9,3,nick,3,0,2,LCI,2,0


In [72]:
dataset = Dataset()
dataset.fit(
    set(cli['id_cliente']), 
    set(prod['id_produto']),
    item_features=cliente_feature_list, 
    user_features=produto_feature_list)

In [73]:
df_merge['cli_prod_id_tuple'] = list(zip(
    df_merge.id_cliente, df_merge.id_produto, df_merge.weight))

In [142]:
df_merge['cli_prod_id_tuple']

0     (1, 1, 0)
1     (1, 2, 1)
2     (1, 3, 0)
3     (1, 4, 0)
4     (2, 1, 0)
5     (2, 2, 1)
6     (2, 3, 0)
7     (2, 4, 1)
8     (3, 1, 1)
9     (3, 2, 0)
10    (3, 3, 0)
11    (3, 4, 1)
12    (4, 1, 1)
13    (4, 2, 1)
14    (4, 3, 1)
15    (4, 4, 0)
Name: cli_prod_id_tuple, dtype: object

In [74]:
interactions, weights = dataset.build_interactions(
    df_merge['cli_prod_id_tuple'])

In [150]:
interactions.shape

(4, 4)

In [75]:
cli['cli_features'] = create_features(
    cli, ['perfil'], 
    'id_cliente')

prod['prod_features'] = create_features(
    prod,
    ['risco'],
    'id_produto')

In [76]:
cli

,id_cliente,nome,perfil,key,cli_features
0,1,Tom,1,0,"(1, [1])"
1,2,Jack,2,0,"(2, [2])"
2,3,nick,3,0,"(3, [3])"
3,4,juli,5,0,"(4, [5])"


In [77]:
prod

,id_produto,nome,risco,key,prod_features
0,1,CDB,1,0,"(1, [1])"
1,2,LCI,2,0,"(2, [2])"
2,3,LCA,3,0,"(3, [3])"
3,4,FII,5,0,"(4, [5])"


In [78]:
cli_features = dataset.build_item_features(
    cli['cli_features'])

In [79]:
prod_features = dataset.build_user_features(
    prod['prod_features'])

In [99]:
prod['prod_features']

0    (1, [1])
1    (2, [2])
2    (3, [3])
3    (4, [5])
Name: prod_features, dtype: object

In [82]:
model = LightFM(
    no_components=150,
    learning_rate=0.05,
    loss='warp',
    random_state=2019)

model.fit(
    interactions,
    item_features=prod_features,
    user_features=cli_features, sample_weight=weights,
    epochs=5, num_threads=4, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [118]:
prod_features.shape

(4, 8)

In [148]:
df_use_for_prediction = prod

scores = model.predict(0, 
              [0, 1, 2, 3],
              user_features=cli_features, 
              item_features=prod_features
             )

df_use_for_prediction['scores'] = scores
df_use_for_prediction = df_use_for_prediction.sort_values(by='scores', ascending=False)[:8]
print('Recommended Products: ')
display(df_use_for_prediction[['nome', 'risco']])

Recommended Products: 


,nome,risco
3,FII,5
2,LCA,3
1,LCI,2
0,CDB,1
